## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-07-20-55-37 +0000,nypost,"Governor Newsom, Mayor Bass duck angry protest...",https://nypost.com/2026/01/07/us-news/governor...
1,2026-01-07-20-53-55 +0000,nyt,Local Officials Call for Federal Forces to Lea...,https://www.nytimes.com/2026/01/07/us/minneapo...
2,2026-01-07-20-52-32 +0000,bbc,"UK armed forces helped US seizure of tanker, s...",https://www.bbc.com/news/articles/ckglnprzk72o...
3,2026-01-07-20-47-15 +0000,bbc,"MPs would vote on using troops in Ukraine, Sta...",https://www.bbc.com/news/articles/ckgln3lgl3jo...
4,2026-01-07-20-47-00 +0000,wsj,U.S. Population Growth Is Projected to Slow Fu...,https://www.wsj.com/politics/policy/u-s-popula...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2326/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
110,trump,51
52,venezuela,42
95,oil,25
130,new,21
184,greenland,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
344,2026-01-06-22-21-00 +0000,wsj,"The White House said the real crime on Jan. 6,...",https://www.wsj.com/politics/policy/white-hous...,151
240,2026-01-07-10-10-24 +0000,bbc,Trump says Venezuela will be 'turning over' up...,https://www.bbc.com/news/articles/c4grxzxjjd8o...,146
161,2026-01-07-15-31-28 +0000,nyt,Trump Claims Venezuela Will Send U.S. Millions...,https://www.nytimes.com/2026/01/06/business/ve...,142
14,2026-01-07-20-36-19 +0000,nyt,U.S. Details Plan for Venezuela’s Oil Sales Af...,https://www.nytimes.com/2026/01/07/business/en...,139
335,2026-01-06-22-34-09 +0000,nyt,The Trump administration covets Venezuela’s oi...,https://www.nytimes.com/live/2026/01/06/world/...,130


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
344,151,2026-01-06-22-21-00 +0000,wsj,"The White House said the real crime on Jan. 6,...",https://www.wsj.com/politics/policy/white-hous...
186,101,2026-01-07-13-30-05 +0000,nypost,"US forces seize sanctioned, Russia-flagged oil...",https://nypost.com/2026/01/07/us-news/us-force...
19,89,2026-01-07-20-27-33 +0000,nypost,New Minneapolis ICE shooting video shows vehic...,https://nypost.com/2026/01/07/us-news/new-minn...
248,61,2026-01-07-10-00-00 +0000,wsj,The welfare-fraud scandal that led Minnesota G...,https://www.wsj.com/politics/elections/minneso...
10,61,2026-01-07-20-42-33 +0000,nypost,Venezuelan interim leader Delcy Rodríguez fire...,https://nypost.com/2026/01/07/world-news/venez...
183,52,2026-01-07-13-34-42 +0000,cbc,U.S. Secretary of State Rubio says he will mee...,https://www.cbc.ca/news/world/us-greenland-eur...
213,39,2026-01-07-11-42-21 +0000,nypost,‘Squad’ Rep. Ayanna Pressley’s ex-con husband ...,https://nypost.com/2026/01/07/us-news/ayanna-p...
173,38,2026-01-07-14-28-30 +0000,cbc,"Carney visiting China next week to talk trade,...",https://www.cbc.ca/news/politics/carney-to-vis...
193,34,2026-01-07-13-07-01 +0000,nypost,Warner Bros. Discovery rejects Paramount Skyda...,https://nypost.com/2026/01/07/business/warner-...
57,33,2026-01-07-19-30-47 +0000,nypost,California ‘Party mom’ rented mansion for drun...,https://nypost.com/2026/01/07/us-news/los-gato...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
